In this notebook, we will introduce how to perform dimension reduction and score its quality with DASH_Train. 

Please make sure you are running the notebook at the directory of "DASH/tutorial/01_Dimension_Reduction_With_DASH_Train", because we will use relative path in this notebook. 

-----------------------------------------------------------------------------------------------------------------------------------------

Dimension reduction is realized with ../../bin/DASH_Train, a command line tool. The usage is

【method_name】is the method used for dimension reduction, including SplitContrastMAE, PCA, UMAP, IsoMap, AE (AutoEncoder) and EncoderMap. Note that except for PCA, all other methods are trained as a neural network, in order to make them derivable when used for seed hopping. 

【traj_info_file】is the input file describing the trajectory to be reduced. We will discuss about it later. 

【high_dim_type】is the atoms selected to describe your reprersent your trajectory. In out article we use "CA", namely Cα atoms. The program also support "heavy", namely all heavy atoms. 

【c_con】,【c_mae】and 【encoder_type】are three parameters only available when method_name == SplitContrast MAE. 【c_con】and【c_mae】are the coefficient in the loass function, namely total_loss = 【c_con】* Contrastive_Loss + 【c_mae】* MAE_Loss. Default is both 1. 【encoder_type】is the encoder used to construct neural networks. Currently support MLP, GCN, GAT and EGNN. MLP by default.

-------------------------------------------------------------------------------------------------------------------------------

The following is an example of 【traj_info_file】, which describes a trajectory of trp_cage. This file is stored in the current directory as trp_cage.in. trp_cage will also be the example in this tutorial.

【name】:    Name of the tajectory, used to create the working directory.

【dir】:     Folder to stroe the pdb file and trajectory file. The calculated pair-wise RMSDs will be stored in this folder so that if you want to rerun dimension
         reduction, the pair-wise RMSDs could directly br read.

【pdb】:     The PDB file, namely topology file for the trjectory.

【dcd】:     The trjaectory file for diemnsion reduction. Just like its name, it should be a DCD file.

【sel_str】: Prody style selection string (http://www.bahargroup.org/prody/manual/reference/atomic/select.html) to select the atoms for alignment before dimension 
         reduction.
         
【step】:    Step to read the DCD file. 

【n_train】: Training neural networks could be arbitary. Therefore, we parallely train【n_train】networks and choose the best one based on loss.

【n_proc】 : Number of processors during parallel training. 

-----------------------

Ok now let's perform dimension reduction!

In [ ]:
import os
os.system("export CUDA_VISIBLE_DEVICES=1;../../bin/DASH_Train SplitContrastMAE trp_cage.in CA")

The command results in a folder named "Train_SplitContrastMAE_trp_cage_pro_DES". In this folder are some important files:

【z.npy】: Calculated 2-D collective variables (CVs), numpy array, n_frame*2.

【countour.jpg】: Free energy surface on CV space.

【hist.jpg】: 2-D Histogram on CV space.

【model】: Complete neural network for CV calculation. Load it by torch.load.

【CVnet_0.pt】,【CVnet_1.pt】: JIT file compiling encoders for calculating 2 CVs. 

【seedX.pdb】: X = 0~19, 20 marginal structures calculated from CV space.

--------------------------



Finally, we will score the trained neural network. This is realized by ../../bin/Score_Net

Usage:

../../bin/Score_Net  folder_for_scoring  traj_info_file

In [ ]:
import os
os.system("../../bin/Score_Net Train_SplitContrastMAE_trp_cage_pro_DES trp_cage.in")

The scores will be stored in Train_SplitContrastMAE_trp_cage_pro_DES/score.txt